# Cluster Analysis Based on MCI Original

In [1]:
'''
Import key packages
'''
import umap
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from functools import reduce
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans

# debug mode (spits messages verbosely)
DEBUG = 0  # change it to 0 to switch it off

loc="../../../WES/Clinical Data/Raw/"
outdata="../../../WES/Clinical Data/Processed/"

config =dict(\
varsnormalise=['udprs_Disease_Progression','udprs_Age_Diagnosis','cog1_MMSE result',\
'udprs_non_tremor','udprs_tremor','udprs_tremor_ratio','cog1_NART','BDI_BDI Total'],\
nonnormvars=['cog1_Trails A Z score','Lm2_cut','ESS_norm','FOGQ_FOGQ 3','DDE_norm'],\
umap_neigbours=10,umap_comps=2,kmeans_try=10)

varsnormalise=config['varsnormalise']
nonnormvars=config['nonnormvars']

In [2]:
def z_score(col):
    x = (col - col.mean())/col.std(ddof=0)
    return  x

def takeonlyx_sd(df,x=5):

    for var in df.columns:
        if re.search('float|int',str(df[var].dtype)):
            maxvar=df[var].mean()+df[var].std()*5
            minvar=df[var].mean()-df[var].std()*5
            mask_rng=(df[var].between(minvar,maxvar))
            df[var][~mask_rng]=df[var][mask_rng].mean()
           # print(df[var].max())
           # print(df.shape)
    return df

def replacenullsmean(df,dropcols=['SYDID','Date of assessment','max_date']):
    usecols=[col for col in df.columns if col not in dropcols]
    df[usecols]=df[usecols].replace([np.inf, -np.inf], np.nan)
    for col in df[usecols].columns:
        nullmask=(df[col].isna())
        df[col][nullmask]=df[col][~nullmask].mean()
    return df

def umap_reducer(df,vars,neighb=10,comp=5):
    df_out=df.copy()
    
    for col in df_out.columns:
        if df_out[col].dtype!='object':
            df_out[col]=df_out[col].fillna(df_out[col][pd.notnull(df_out[col])].mean())
              
    dmod=df_out[[col for col in vars if col!='SYDID' and re.search('float',str(df_out[col].dtype))
                and df_out[col].count()>=df_out.shape[0]]]
    print(dmod.shape)
    
    reducer= umap.UMAP(n_neighbors=neighb,n_components=comp)
    reddata = pd.DataFrame(reducer.fit_transform(dmod)).reset_index()
    reddata.shape
    
    reddata.columns=['index','umap_var1','umap_var2']
    df_out[['index','umap_var1','umap_var2']]=reddata[['index','umap_var1','umap_var2']]
    return df_out

def elbow_clust(df,clustrange=10,clustfit=4,dropvars=['SYDID','Date of assessment','max_date']):
    df_vars=df.drop(columns=dropvars)
    dv2=[col for col in df_vars if df_vars[col].dtype=='float32']
    print(dv2)
    df_vars=df_vars[dv2]
    print(df_vars.columns)
    df_vars=replacenullsmean(df_vars)
    
    wcss=[]
    #print(df_vars.head())
    for i in range(1,clustrange): 
         kmeans = KMeans(n_clusters=i, init ='k-means++', max_iter=300,  n_init=clustrange-1,random_state=0)
         kmeans.fit(df_vars)
         wcss.append(kmeans.inertia_)
    plt.plot(range(1,clustrange),wcss)
    plt.title('The Elbow Method Graph')
    plt.xlabel('Number of clusters')
    plt.ylabel('WCSS')
    plt.show()
    kmeans = KMeans(n_clusters=clustfit, random_state=0).fit(df_vars)
    df['cluster_label']=kmeans.labels_.tolist()
    return df

In [3]:
PDcombined=pd.read_csv('%s%s' % (outdata,'PDcombined.csv'))

#### Data Processing

In [4]:

logmem2cutoff=PDcombined['cog1_Logical Memory II'].mean()+PDcombined['cog1_Logical Memory II'].std()*1.5
masklm2=(PDcombined['cog1_Logical Memory II']<logmem2cutoff)

PDcombined['Lm2_cut']=0
PDcombined['Lm2_cut'][(masklm2)]=1

mask0=(PDcombined['udprs_DDE']>0)
masklt1000=(PDcombined['udprs_DDE']<1000)
PDcombined['DDE_norm']=0
PDcombined['DDE_norm'][mask0&masklt1000]=1
PDcombined['DDE_norm'][mask0&~masklt1000]=2

maskbdi=(PDcombined['BDI_BDI Total']>13)
PDcombined['BDI_norm']=0
PDcombined['BDI_norm'][maskbdi]=1

maskess=(PDcombined['ESS_ESS Total']>10)
PDcombined['ESS_norm']=0
PDcombined['ESS_norm'][maskess]=1

mask_ratio=((PDcombined['udprs_tremor_ratio']==np.inf))
PDcombined['udprs_tremor_ratio'][mask_ratio]=PDcombined['udprs_tremor_ratio'][~mask_ratio].mean()

for col in varsnormalise:
    nullmask=(PDcombined[col].isna())
    PDcombined[col][nullmask]=PDcombined[col][~nullmask].mean()
    PDcombined[col+'_z_score']=z_score(PDcombined[col])
vars_z=[var +'_z_score' for var in varsnormalise ]
varscluster=np.append(['SYDID','Date of assessment'],nonnormvars)
varscluster=np.append(varscluster,vars_z)

'''
Substitute to only have max date
'''
PDcombined['max_date']=PDcombined.groupby('SYDID')['Date of assessment'].transform(max)

mask_date_max=(PDcombined['max_date']==PDcombined['Date of assessment'])
PDcombined_sub=PDcombined[mask_date_max]

'''
Removal of columns that don't work
'''

othcols=[col for col in PDcombined.columns if (PDcombined[col].dtype!='object'\
and col not in varscluster) or col in ['SYDID','Date of assessment','udprs_Disease_Progression',\
'udprs_Age_Diagnosis']]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path(

In [5]:
PDcluster_alt=PDcombined_sub[othcols]
PDcluster=PDcombined_sub[varscluster]
for var in PDcluster.columns:
    PDcluster=PDcluster[(PDcluster[var]!=999)]
    
PDcluster_alt=takeonlyx_sd(df=PDcluster_alt,x=3)
PDcluster_alt=replacenullsmean(PDcluster_alt)

'''
Reduce using umap
'''

pdclust_umap=umap_reducer(PDcluster_alt,PDcluster_alt.columns,\
neighb=config['umap_neigbours'],comp=config['umap_comps'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:9114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/pandas/core/ge

(175, 334)


In [ ]:
dmod=df_out[[col for col in vars if col!='SYDID' and re.search('float',str(df_out[col].dtype))
                and df_out[col].count()>=df_out.shape[0]]]
print(dmod.shape)

reducer= umap.UMAP(n_neighbors=neighb,n_components=comp)
reddata = pd.DataFrame(reducer.fit_transform(dmod)).reset_index()
reddata.shape

reddata.columns=['index','umap_var1','umap_var2']
df_out[['index','umap_var1','umap_var2']]=reddata[['index','umap_var1','umap_var2']]

In [ ]:
pdclust_umap=umap_reducer(PDcluster,PDcluster.columns,neighb=10,comp=2)

In [ ]:
PDcluster_clust=elbow_clust(pdclust_umap[['SYDID','Date of assessment','umap_var1','umap_var2']],\
clustfit=2,dropvars=['SYDID','Date of assessment'])

sns.scatterplot(data=PDcluster_clust, x="umap_var1", y="umap_var2",hue='cluster_label')
